In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr

# Initialize SparkSession correctly
spark = SparkSession.builder \
    .appName("PracticeProject") \
    .enableHiveSupport() \
    .getOrCreate()

# Customers Data
customers_data = [
    (101, 'Ali', 'ali@gmail.com', 'Mumbai', '2022-05-10'),
    (102, 'Neha', 'neha@yahoo.com', 'Delhi', '2023-01-15'),
    (103, 'Ravi', 'ravi@hotmail.com', 'Bangalore', '2021-11-01'),
    (104, 'Sneha', 'sneha@outlook.com', 'Hyderabad', '2020-07-22'),
    (105, 'Amit', 'amit@gmail.com', 'Chennai', '2023-03-10'),
]

# Orders Data
orders_data = [
    (1, 101, 'Laptop', 'Electronics', 2, 50000.0, '2024-01-10'),
    (2, 101, 'Mouse', 'Electronics', 1, 1200.0, '2024-01-15'),
    (3, 102, 'Tablet', 'Electronics', 1, 20000.0, '2024-02-01'),
    (4, 103, 'Bookshelf', 'Furniture', 1, 3500.0, '2024-02-10'),
    (5, 104, 'Mixer', 'Appliances', 1, 5000.0, '2024-02-15'),
    (6, 105, 'Notebook', 'Stationery', 5, 500.0, '2024-03-01'),
    (7, 102, 'Phone', 'Electronics', 1, 30000.0, '2024-03-02'),
]

# Create DataFrames
customers_df = spark.createDataFrame(customers_data, ["CustomerID", "Name", "Email", "City", "SignupDate"])
orders_df = spark.createDataFrame(orders_data, ["OrderID", "CustomerID", "Product", "Category", "Quantity", "Price", "OrderDate"])
# Create the database if not exists
spark.sql("CREATE DATABASE IF NOT EXISTS sales")

# Save the DataFrames as tables in the sales database
customers_df.write.mode("overwrite").saveAsTable("sales.customers")
orders_df.write.mode("overwrite").saveAsTable("sales.orders")

# SECTION A: PySpark DataFrame Tasks


In [7]:
# Q1. Add a column TotalAmount = Price * Quantity
orders_df = orders_df.withColumn("TotalAmount", col("Price") * col("Quantity"))
orders_df.show()

# Q2. Filter all orders with TotalAmount > 10000
orders_df.filter(col("TotalAmount") > 10000).show()

# Q3. Standardize the City field in customers_df (e.g., lowercase)
customers_df = customers_df.withColumn("City", expr("lower(City)"))
customers_df.show()

# Q4. Extract year from OrderDate and add a new column OrderYear
orders_df = orders_df.withColumn("OrderYear", year("OrderDate"))
orders_df.show()

# Q5. Fill null values in any column of your choice with defaults (Category)
orders_df = orders_df.fillna({"Category": "Unknown"})
orders_df.show()

# Q6. Use when/otherwise to categorize orders
orders_df = orders_df.withColumn(
    "OrderCategory",
    when(col("TotalAmount") < 5000, "Low")
    .when((col("TotalAmount") >= 5000) & (col("TotalAmount") <= 20000), "Medium")
    .otherwise("High")
)
orders_df.select("OrderID", "TotalAmount", "OrderCategory").show()


+-------+----------+---------+-----------+--------+-------+----------+-----------+
|OrderID|CustomerID|  Product|   Category|Quantity|  Price| OrderDate|TotalAmount|
+-------+----------+---------+-----------+--------+-------+----------+-----------+
|      1|       101|   Laptop|Electronics|       2|50000.0|2024-01-10|   100000.0|
|      2|       101|    Mouse|Electronics|       1| 1200.0|2024-01-15|     1200.0|
|      3|       102|   Tablet|Electronics|       1|20000.0|2024-02-01|    20000.0|
|      4|       103|Bookshelf|  Furniture|       1| 3500.0|2024-02-10|     3500.0|
|      5|       104|    Mixer| Appliances|       1| 5000.0|2024-02-15|     5000.0|
|      6|       105| Notebook| Stationery|       5|  500.0|2024-03-01|     2500.0|
|      7|       102|    Phone|Electronics|       1|30000.0|2024-03-02|    30000.0|
+-------+----------+---------+-----------+--------+-------+----------+-----------+

+-------+----------+-------+-----------+--------+-------+----------+-----------+
|Orde

#SECTION B: Spark SQL Tasks

In [8]:
# Register Temp Views for SQL
orders_df.createOrReplaceTempView("orders")
customers_df.createOrReplaceTempView("customers")

# Q7. Run a SQL query to list all orders made by “Ali”
spark.sql("""
    SELECT * FROM orders
    WHERE CustomerID = (SELECT CustomerID FROM customers WHERE Name = 'Ali')
""").show()

# Q8. Get total spending by each customer using SQL
spark.sql("""
    SELECT o.CustomerID, SUM(o.TotalAmount) AS TotalSpending
    FROM orders o
    GROUP BY o.CustomerID
""").show()

# Q9. Find out which category made the highest total revenue
spark.sql("""
    SELECT Category, SUM(TotalAmount) AS Revenue
    FROM orders
    GROUP BY Category
    ORDER BY Revenue DESC
    LIMIT 1
""").show()

# Q10. Create a view customer_orders showing CustomerName, Product, TotalAmount
spark.sql("""
    CREATE OR REPLACE TEMP VIEW customer_orders AS
    SELECT c.Name AS CustomerName, o.Product, o.TotalAmount
    FROM customers c
    JOIN orders o ON c.CustomerID = o.CustomerID
""")

# Q11. Query the view for products ordered after Feb 2024
spark.sql("""
    SELECT * FROM customer_orders
    JOIN orders o ON customer_orders.Product = o.Product
    WHERE o.OrderDate > '2024-02-28'
""").show()


+-------+----------+-------+-----------+--------+-------+----------+-----------+---------+-------------+
|OrderID|CustomerID|Product|   Category|Quantity|  Price| OrderDate|TotalAmount|OrderYear|OrderCategory|
+-------+----------+-------+-----------+--------+-------+----------+-----------+---------+-------------+
|      1|       101| Laptop|Electronics|       2|50000.0|2024-01-10|   100000.0|     2024|         High|
|      2|       101|  Mouse|Electronics|       1| 1200.0|2024-01-15|     1200.0|     2024|          Low|
+-------+----------+-------+-----------+--------+-------+----------+-----------+---------+-------------+

+----------+-------------+
|CustomerID|TotalSpending|
+----------+-------------+
|       101|     101200.0|
|       102|      50000.0|
|       103|       3500.0|
|       104|       5000.0|
|       105|       2500.0|
+----------+-------------+

+-----------+--------+
|   Category| Revenue|
+-----------+--------+
|Electronics|151200.0|
+-----------+--------+

+--------

#SECTION C: Advanced Practice



In [9]:
# Q12. Create a Global Temp View from customers_df
customers_df.createOrReplaceGlobalTempView("customers")
spark.sql("SELECT * FROM global_temp.customers WHERE City = 'mumbai'").show()

# Q13. Save the transformed orders_df (with TotalAmount) to a Parquet file
orders_df.write.mode("overwrite").parquet("/tmp/orders_parquet")

# Q14. Read back the Parquet file and count how many orders are in it
read_df = spark.read.parquet("/tmp/orders_parquet")
print("Total Orders:", read_df.count())


+----------+----+-------------+------+----------+
|CustomerID|Name|        Email|  City|SignupDate|
+----------+----+-------------+------+----------+
|       101| Ali|ali@gmail.com|mumbai|2022-05-10|
+----------+----+-------------+------+----------+

Total Orders: 7


#SECTION D: UDF + Built-in Function Tasks

In [10]:
# Q15. UDF to mask emails
def mask_email(email):
    user, domain = email.split('@')
    return user[0] + "***@" + domain

mask_udf = udf(mask_email, StringType())
customers_df = customers_df.withColumn("MaskedEmail", mask_udf(col("Email")))
customers_df.select("Email", "MaskedEmail").show()

# Q16. Use concat_ws() to create label like 'Ali from Mumbai'
customers_df = customers_df.withColumn("Label", concat_ws(" from ", col("Name"), col("City")))
customers_df.select("Label").show()

# Q17. Use regexp_replace() to remove special characters from product names
orders_df = orders_df.withColumn("CleanProduct", regexp_replace("Product", "[^a-zA-Z0-9]", ""))
orders_df.select("Product", "CleanProduct").show()

# Q18. Use to_date() and datediff() to calculate customer age in days
customers_df = customers_df.withColumn("SignupDate", to_date("SignupDate"))
today_str = date.today().strftime('%Y-%m-%d')
customers_df = customers_df.withColumn("AgeInDays", datediff(lit(today_str), col("SignupDate")))
customers_df.select("Name", "SignupDate", "AgeInDays").show()


+-----------------+----------------+
|            Email|     MaskedEmail|
+-----------------+----------------+
|    ali@gmail.com|  a***@gmail.com|
|   neha@yahoo.com|  n***@yahoo.com|
| ravi@hotmail.com|r***@hotmail.com|
|sneha@outlook.com|s***@outlook.com|
|   amit@gmail.com|  a***@gmail.com|
+-----------------+----------------+

+--------------------+
|               Label|
+--------------------+
|     Ali from mumbai|
|     Neha from delhi|
| Ravi from bangalore|
|Sneha from hyderabad|
|   Amit from chennai|
+--------------------+

+---------+------------+
|  Product|CleanProduct|
+---------+------------+
|   Laptop|      Laptop|
|    Mouse|       Mouse|
|   Tablet|      Tablet|
|Bookshelf|   Bookshelf|
|    Mixer|       Mixer|
| Notebook|    Notebook|
|    Phone|       Phone|
+---------+------------+

+-----+----------+---------+
| Name|SignupDate|AgeInDays|
+-----+----------+---------+
|  Ali|2022-05-10|     1121|
| Neha|2023-01-15|      871|
| Ravi|2021-11-01|     1311|
|Sneha|2